In [1]:
import os
import sys
import yaml
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
from model.fp.nnfp import get_fingerprinter
import numpy as np

In [2]:
def load_config(config_fname):
    config_filepath = './config/' + config_fname + '.yaml'
    if os.path.exists(config_filepath):
        print(f'cli: Configuration from {config_filepath}')
    else:
        sys.exit(f'cli: ERROR! Configuration file {config_filepath} is missing!!')

    with open(config_filepath, 'r') as f:
        cfg = yaml.safe_load(f)
    return cfg

In [3]:
config:str = "default"  
cfg = load_config(config)

checkpoint_root_dir = '/mnt/dev/rodrigoalmeida/NeuralNetworks/neural-audio-fp/logs/checkpoint/'
checkpoint_name:str = 'CHECKPOINT_BSZ_120'
checkpoint_index:int = 100

cli: Configuration from ./config/default.yaml


In [4]:
def load_checkpoint(checkpoint_root_dir, checkpoint_name, checkpoint_index,
                    m_fp):
    """ Load a trained fingerprinter """
    # Create checkpoint
    checkpoint = tf.train.Checkpoint(model=m_fp)
    checkpoint_dir = checkpoint_root_dir + f'/{checkpoint_name}/'
    c_manager = tf.train.CheckpointManager(checkpoint, checkpoint_dir,
                                           max_to_keep=None)

    # Load
    if checkpoint_index == None:
        tf.print("\x1b[1;32mArgument 'checkpoint_index' was not specified.\x1b[0m")
        tf.print('\x1b[1;32mSearching for the latest checkpoint...\x1b[0m')
        latest_checkpoint = c_manager.latest_checkpoint
        if latest_checkpoint:
            checkpoint_index = int(latest_checkpoint.split(sep='ckpt-')[-1])
            status = checkpoint.restore(latest_checkpoint)
            status.expect_partial()
            tf.print(f'---Restored from {c_manager.latest_checkpoint}---')
        else:
            raise FileNotFoundError(f'Cannot find checkpoint in {checkpoint_dir}')
    else:
        checkpoint_fpath = checkpoint_dir + 'ckpt-' + str(checkpoint_index)
        status = checkpoint.restore(checkpoint_fpath) # Let TF to handle error cases.
        status.expect_partial()
        tf.print(f'---Restored from {checkpoint_fpath}---')
    return checkpoint_index

In [35]:
checkpoint = tf.train.Checkpoint(model=meuModelo)
c_manager = tf.train.CheckpointManager(checkpoint, checkpoint_root_dir+checkpoint_name, max_to_keep=None)

latest_checkpoint = c_manager.latest_checkpoint
latest_checkpoint
checkpoint_index = int(latest_checkpoint.split(sep='ckpt-')[-1])
checkpoint_fpath =  checkpoint_root_dir+checkpoint_name + 'ckpt-' + str(checkpoint_index)
status = checkpoint.restore(checkpoint_fpath)

In [39]:
status

In [8]:
meuModelo = get_fingerprinter(cfg, trainable=False)

#checkpoint_index = load_checkpoint(checkpoint_root_dir, checkpoint_name, 100, meuModelo)

In [9]:
x = tf.constant(np.random.randn(1,256,32,1), dtype=tf.float32) # BxFxTx1

In [10]:
emb_1s = meuModelo(x)

In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input

def convlayer(hidden_ch=128,
              strides=[(1,1),(1,1)],
              norm='layer_norm2d'):
    conv2d_1x3 = tf.keras.layers.Conv2D(hidden_ch,
                                        kernel_size=(1, 3),
                                        strides=strides[0],
                                        padding='SAME',
                                        dilation_rate=(1, 1),
                                        kernel_initializer='glorot_uniform',
                                        bias_initializer='zeros')
    conv2d_3x1 = tf.keras.layers.Conv2D(hidden_ch,
                                        kernel_size=(3, 1),
                                        strides=strides[1],
                                        padding='SAME',
                                        dilation_rate=(1, 1),
                                        kernel_initializer='glorot_uniform',
                                        bias_initializer='zeros')
    if norm == 'layer_norm1d':
        BN_1x3 = tf.keras.layers.LayerNormalization(axis=-1)
        BN_3x1 = tf.keras.layers.LayerNormalization(axis=-1)
    elif norm == 'layer_norm2d':
        BN_1x3 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
        BN_3x1 = tf.keras.layers.LayerNormalization(axis=(1, 2, 3))
    else:
        BN_1x3 = tf.keras.layers.BatchNormalization(axis=-1)
        BN_3x1 = tf.keras.layers.BatchNormalization(axis=-1)
        
    forward = tf.keras.Sequential([conv2d_1x3,
                                   tf.keras.layers.ELU(),
                                   BN_1x3,
                                   conv2d_3x1,
                                   tf.keras.layers.ELU(),
                                   BN_3x1
                                   ])
    
    return forward



def create_sequential_front_conv(input_shape=(256,32,1),
                                 emb_sz=128,
                                 front_hidden_ch=[128, 128, 256, 256, 512, 512, 1024, 1024],
                                 front_strides=[[(1,2), (2,1)], [(1,2), (2,1)],
                                                [(1,2), (2,1)], [(1,2), (2,1)],
                                                [(1,1), (2,1)], [(1,2), (2,1)],
                                                [(1,1), (2,1)], [(1,2), (2,1)]],
                                 norm='layer_norm2d'):
    front_conv = tf.keras.Sequential(name='ConvLayers')
    if ((front_hidden_ch[-1] % emb_sz) != 0):
        front_hidden_ch[-1] = ((front_hidden_ch[-1]//emb_sz) + 1) * emb_sz

    for i in range(len(front_strides)):
        front_conv.add(convlayer(hidden_ch=front_hidden_ch[i], strides=front_strides[i], norm=norm))
    front_conv.add(tf.keras.layers.Flatten())

    return front_conv



def auxiliar(input1):
    conv_layer = create_sequential_front_conv(input_shape=(256,32,1),
                                               emb_sz=128,
                                               front_hidden_ch=[128, 128, 256, 256, 512, 512, 1024, 1024],
                                               front_strides=[[(1,2), (2,1)], [(1,2), (2,1)],
                                                              [(1,2), (2,1)], [(1,2), (2,1)],
                                                              [(1,1), (2,1)], [(1,2), (2,1)],
                                                              [(1,1), (2,1)], [(1,2), (2,1)]],
                                               norm='layer_norm2d')

    unit_dim = [32, 1]
    q = 128
    arquiteturas_densas = tf.keras.Sequential([tf.keras.layers.Dense(unit_dim[0], activation='elu'),
                                               tf.keras.layers.Dense(unit_dim[1])])

    x = input1
    #x reshape
    x = conv_layer(x)

    y_list = [0] * q
    x_split = tf.split(x, num_or_size_splits=128, axis=1)

    for v, k in enumerate(x_split):
        y_list[v] = arquiteturas_densas(k)

    out = tf.concat(y_list, axis=1)
    output = tf.math.l2_normalize(out, axis=1)
    return output


def get_fingerprinting(input1):
    output = auxiliar(input1)
    fingerprinting_model = Model(inputs=input1, outputs=output)
    return fingerprinting_model

In [ ]:
functional_model=get_fingerprinting(x)
functional_model.set_weights(meuModelo.get_weights())

Pesos

In [8]:
dirWeights = '/home/rodrigo/Documents/neural-audio-fp/ModeloGuardadoFolder/'
#module_no_signatures_path = os.path.join(tmpdir, 'module_no_signatures')

In [ ]:
tf.saved_model.save(meuModelo, dirWeights)

In [ ]:
meuModelo.save_weights('/home/rodrigo/Documents/neural-audio-fp/ModeloGuardadoFolder/')

In [46]:
v=meuModelo.front_conv.layers[0].conv2d_1x3.kernel

In [ ]:
v

### Summary()

In [28]:
meuModelo.summary()

Model: "finger_printer"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ConvLayers (Sequential)     (2, 1024)                 16897920  
                                                                 
 div_enc_layer (DivEncLayer  multiple                  41088     
 )                                                               
                                                                 
Total params: 16939008 (64.62 MB)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 16939008 (64.62 MB)
_________________________________________________________________


In [31]:
meuModelo.front_conv.summary()

Model: "ConvLayers"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_layer (ConvLayer)      (2, 128, 16, 128)         1622656   
                                                                 
 conv_layer_1 (ConvLayer)    (2, 64, 8, 128)           491776    
                                                                 
 conv_layer_2 (ConvLayer)    (2, 32, 4, 256)           492032    
                                                                 
 conv_layer_3 (ConvLayer)    (2, 16, 2, 256)           442880    
                                                                 
 conv_layer_4 (ConvLayer)    (2, 8, 2, 512)            1229824   
                                                                 
 conv_layer_5 (ConvLayer)    (2, 4, 1, 512)            1586176   
                                                                 
 conv_layer_6 (ConvLayer)    (2, 2, 1, 1024)           4

In [ ]:
meuModelo.div_enc.weights

In [45]:
meuModelo.div_enc

0